Combinatorial Auction Mechanism for Computation Distribution::

Problem Statement: 
    Given a range of available resources on a computational cluster (divided into 3 types: processing, memory, 
    disk space, each with a specific amount), implement an efficient (VCG) auction mechanism that can distribute 
    these to N bidding agents that each want to schedule a randomly constructed job (i.e. with random resource 
    requirements) with a true valuation that is additive in terms of the values of the required resources.

In [19]:
from IPython.display import HTML

input_form = """
<div style="background-color:gainsboro; border:solid black;">
<div><b>Enter the number of items and bidders for the Auction:</b></div>
<div style="display:inline">
Players: <input type="text" id="var_players" value="1">
Items  : <input type="text" id="var_items" value="2">
</div>
<button onclick="gen_table()">Generate Table</button>
</div>
<style>
    table{
        width:100%;
        height:10)%;
    }
    table td{
        padding:5px;
        margin:5px;
        border:1px solid #ccc;
    }
</style>
<div id="box">
<table id="basicTable">
</table>
</div>
<button onclick="rescale()">Scale Values</button>
<button onclick="set_valuations()">Set Values</button>
<button onclick="reset_valuations()">Reset Values</button>
"""



javascript = """
<script type="text/Javascript">
    function gen_table(){
        var players = document.getElementById('var_players').value;
        var items = document.getElementById('var_items').value;
        
        var parent = document.getElementById('box');
        var child = document.getElementById('basicTable');
        parent.removeChild(child);
        mytable = $('<table></table>').attr({ id: "basicTable" });
        var rows = players;
        var cols = items;
        var tr = [];
        var row = $('<tr><td></td></tr>').attr({ class: ["class1"].join(' ') }).appendTo(mytable);
        for (var j = 0; j < cols; j++) {
                $("<td>Item "+j+"</td>").appendTo(row); 
            }
        for (var i = 0; i < rows; i++) {
            var row = $('<tr></tr>').attr({ class: ["class1"].join(' ') }).appendTo(mytable);
            $("<td>Player "+i+"</td>").appendTo(row)
            for (var j = 0; j < cols; j++) {
                $("<td><input type='text' id='val_"+i+"_"+j+"' value='1'></td>").appendTo(row); 
            }

        }
        console.log("TTTTT:"+mytable.html());
        mytable.appendTo("#box"); 
    }

    function set_valuations(){
        var players = document.getElementById('var_players').value;
        var items = document.getElementById('var_items').value;
        var val = [];
        var x;
        for (var i = 0; i < players; i++) {
            val[i] = [];
            for (var j = 0; j < items; j++) {
                x = 'val_'+i+'_'+j;
                val[i][j] = document.getElementById(x).value;
            }
        }
        
        var kernel = IPython.notebook.kernel;
        var cmd1 = "players = " + players;
        var cmd2 = "items = " + items;
        var cmd3 = "my_val = " + val;
        
        console.log("Executing Command: " + cmd1);     
        kernel.execute(cmd1);
        
        console.log("Executing Command: " + cmd2);
        kernel.execute(cmd2);  
        
        console.log("Executing Command: " + cmd3);
        kernel.execute(cmd3); 

    }
    
    function rescale(){
        var players = document.getElementById('var_players').value;
        var items = document.getElementById('var_items').value;
        var val = [];
        var sum = 0;
        var x;
        for (var i = 0; i < players; i++) {
            val[i] = [];
            for (var j = 0; j < items; j++) {
                x = 'val_'+i+'_'+j;
                val[i][j] = parseInt(document.getElementById(x).value);
                sum = sum + val[i][j];  
            }
            for (var j = 0; j < items; j++) {
                val[i][j] = val[i][j]*100/sum;
                x = 'val_'+i+'_'+j;
                document.getElementById(x).value = val[i][j];
            }
            sum = 0
        }        
    }
    function reset_valuations(){
        var players = document.getElementById('var_players').value;
        var items = document.getElementById('var_items').value;
        var val = [];
        var x;
        for (var i = 0; i < players; i++) {
            val[i] = [];
            for (var j = 0; j < items; j++) {
                x = 'val_'+i+'_'+j;
                document.getElementById(x).value = 1;
            }
        }
    
    }
</script>
"""

HTML(input_form + javascript)

In [20]:
# First compute all different one-to-one allocations of items to bidders. 
import numpy as np
import itertools

k = 0
allocs = np.zeros((items, players, 2),dtype=int)
valuations = np.zeros((players, items),dtype=int)
for i in range(0,players):
    for j in range(0,items):
        allocs[j,i] = [j,i]
        valuations[i,j] = my_val[k]
        k += 1
print('valuations are :: ',list(valuations))

valuations are ::  [array([20, 30, 10]), array([30, 20, 20])]


In [21]:
# Identify all possible allocations of items to bidders
x = list(itertools.product(*allocs))
#for i in x:
#    print(i[0])
#    print(i[1])
r = len(x)
c = len(x[0])
p = np.zeros((r,items+players+1),dtype=int)

for i in range(0,r):
    for j in range(0,c):
        plid = x[i][j][1]+items # initial columns are items
        p[i,plid] = p[i,plid] + valuations[x[i][j][1], x[i][j][0]]
        p[i,x[i][j][0]] = x[i][j][1]
    p[i,-1] = np.sum(p[i,-players-1:-1])
print(p)

[[ 0  0  0 60  0 60]
 [ 0  0  1 50 20 70]
 [ 0  1  0 30 20 50]
 [ 0  1  1 20 40 60]
 [ 1  0  0 40 30 70]
 [ 1  0  1 30 50 80]
 [ 1  1  0 10 50 60]
 [ 1  1  1  0 70 70]]


In [22]:
# Store the data in the form of a Dataframe
import pandas as pd

auctions = pd.DataFrame(p)
auctions.columns = ['i'+ str(i) for i in range(0,items)] + \
                      ['v'+ str(i) for i in range(0,players)] + \
                      ['v_tot']
print(auctions.columns)
print(auctions.values)
v_max_without = np.zeros(players,dtype=int)
for i in range(0,players):
    v_max_without[i] = max(auctions.v_tot[(auctions.filter(regex='i') != i).all(1)])
    print(v_max_without[i],i)

Index(['i0', 'i1', 'i2', 'v0', 'v1', 'v_tot'], dtype='object')
[[ 0  0  0 60  0 60]
 [ 0  0  1 50 20 70]
 [ 0  1  0 30 20 50]
 [ 0  1  1 20 40 60]
 [ 1  0  0 40 30 70]
 [ 1  0  1 30 50 80]
 [ 1  1  0 10 50 60]
 [ 1  1  1  0 70 70]]
70 0
60 1


In [23]:
print(auctions)
vcg = auctions[auctions.v_tot == max(auctions.v_tot)].copy().reset_index(drop=True)
print(vcg)
for i in range(0,players):
    print(v_max_without[i] - (vcg.v_tot - vcg['v'+str(i)]))
    #Clarke-PivotRule 
    #(social welfare of others if i were absent) - (social welfare of others when i is present).
    vcg.loc[:,'p'+str(i)] = v_max_without[i] - (vcg.v_tot - vcg['v'+str(i)])

   i0  i1  i2  v0  v1  v_tot
0   0   0   0  60   0     60
1   0   0   1  50  20     70
2   0   1   0  30  20     50
3   0   1   1  20  40     60
4   1   0   0  40  30     70
5   1   0   1  30  50     80
6   1   1   0  10  50     60
7   1   1   1   0  70     70
   i0  i1  i2  v0  v1  v_tot
0   1   0   1  30  50     80
0    20
dtype: int32
0    30
dtype: int32


In [25]:
col_names = list(vcg)
print('Items: ',[s for s in col_names if 'i' in s[0]])
print('Valuation of Player to their allocation: ',[s for s in col_names if 'v' in s[0] and '_' not in s])
print('Payment of Player for their allocation: ',[s for s in col_names if 'p' in s[0]])
HTML(vcg.to_html())

Items:  ['i0', 'i1', 'i2']
Valuation of Player to their allocation:  ['v0', 'v1']
Payment of Player for their allocation:  ['p0', 'p1']


,i0,i1,i2,v0,v1,v_tot,p0,p1
0,1,0,1,30,50,80,20,30


In [27]:
print('VCG Payments:\n')
for i in range(0,players):
    print('VCG Cost for Player ', i, ': ', vcg['v'+str(i)][0] - vcg['p'+str(i)][0])
    print(vcg['v'+str(i)][0],'-',vcg['p'+str(i)][0])

VCG Payments:

VCG Cost for Player  0 :  10
30 - 20
VCG Cost for Player  1 :  20
50 - 30
